<h1 style="border-radius: 10px; border: 2px solid #6B8E23; background-color: #F5F5DC; font-family: 'Pacifico', cursive; font-size: 200%; text-align: center; border-radius: 15px 50px; padding: 15px; box-shadow: 5px 5px 5px #556B2F; color: #556B2F;">🐼 Social Proof of Trending Videos 🐼</h1>

<a id = "1"></a><br>
<p style="font-family: 'Pacifico', cursive; font-weight: bold; letter-spacing: 2px; color: #556B2F; font-size: 160%; text-align: left; padding: 0px; border-bottom: 3px solid">✨Import Libraries✨</p>

In [1]:
import numpy as np
import pandas as pd
import math
import scipy.stats as st
from sklearn.preprocessing import MinMaxScaler

import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=UserWarning)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [2]:
df = pd.read_csv("/kaggle/input/youtube-new/USvideos.csv", usecols=["video_id","views","likes","dislikes","comment_count"])
df.head()

,video_id,views,likes,dislikes,comment_count
0,2kyS6SvSYSE,748374,57527,2966,15954
1,1ZAPwfrtAFY,2418783,97185,6146,12703
2,5qpjK5DgCt4,3191434,146033,5339,8181
3,puqaWrEC7tY,343168,10172,666,2146
4,d380meD0W0M,2095731,132235,1989,17518


<a id = "2"></a><br>
<p style="font-family: 'Pacifico', cursive; font-weight: bold; letter-spacing: 2px; color: #556B2F; font-size: 160%; text-align: left; padding: 0px; border-bottom: 3px solid">✨Looking Data✨</p>

In [3]:
df.shape

(40949, 5)

In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
views,40949.0000,2360784.6383,7394113.7597,549.0000,242329.0000,681861.0000,1823157.0000,225211923.0000
likes,40949.0000,74266.7024,228885.3382,0.0000,5424.0000,18091.0000,55417.0000,5613827.0000
dislikes,40949.0000,3711.4009,29029.7059,0.0000,202.0000,631.0000,1938.0000,1674420.0000
comment_count,40949.0000,8446.8037,37430.4870,0.0000,614.0000,1856.0000,5755.0000,1361580.0000


In [5]:
df.duplicated().sum()

48

In [6]:
df.drop_duplicates(inplace=True)

In [7]:
df.sort_values("views", ascending=False).head(15)

,video_id,views,likes,dislikes,comment_count
38547,VYOjWnS4cMY,225211923,5023450,343541,517232
38345,VYOjWnS4cMY,220490543,4962403,338105,512337
38146,VYOjWnS4cMY,217750076,4934188,335462,509799
37935,VYOjWnS4cMY,210338856,4836448,326902,501722
37730,VYOjWnS4cMY,205643016,4776680,321493,496211
37531,VYOjWnS4cMY,200820941,4714942,316129,491005
37333,VYOjWnS4cMY,196222618,4656929,311042,485797
37123,VYOjWnS4cMY,190950401,4594931,305435,479917
36913,VYOjWnS4cMY,184446490,4512326,298157,473039
36710,VYOjWnS4cMY,179045286,4437175,291098,466470


In [8]:
df = df.groupby("video_id").agg({"views":"max","likes":"max","dislikes":"max","comment_count":"max"})
df.sort_values("views", ascending=False).head(15)

,views,likes,dislikes,comment_count
video_id,,,,
VYOjWnS4cMY,225211923,5023450,343541,517232
FlsCjmMhFmw,149376127,3093544,1643059,827755
ffxKSjUwKdU,148689896,3094021,129502,242039
zEf423kYfqk,139334502,1425496,119798,83941
7C2z4GqqS5E,123010920,5613827,206892,1228655
M4ZoCHID9GI,122544931,1427436,40837,55320
TyHvyGVs42U,102012605,2376636,117196,134224
xTlNMmZKwpA,94254507,1816753,102474,101077
6ZfuNTqbHE8,91933007,2625661,53709,350458


<a id = "3"></a><br>
<p style="font-family: 'Pacifico', cursive; font-weight: bold; letter-spacing: 2px; color: #556B2F; font-size: 160%; text-align: left; padding: 0px; border-bottom: 3px solid">✨Sorting Reviews✨</p>

In [9]:
def score_likes_dislikes_diff(like, dislike):
    
    return like - dislike

In [10]:
df["score_likes_dislikes_diff"] = df.apply(lambda x: score_likes_dislikes_diff(x["likes"],x["dislikes"]), axis=1)

In [11]:
df.sort_values("score_likes_dislikes_diff", ascending=False).head(15)

,views,likes,dislikes,comment_count,score_likes_dislikes_diff
video_id,,,,,
7C2z4GqqS5E,123010920,5613827,206892,1228655,5406935
VYOjWnS4cMY,225211923,5023450,343541,517232,4679909
ffxKSjUwKdU,148689896,3094021,129502,242039,2964519
kTlv5_Bs8aw,36857298,2729292,47896,546100,2681396
p8npDG2ulKQ,29741771,2700800,29341,371864,2671459
OK3GJ0WIQ8s,23416810,2672431,29088,477233,2643343
6ZfuNTqbHE8,91933007,2625661,53709,350458,2571952
aJOTlE1K90k,66529577,2488565,43464,142410,2445101
TyHvyGVs42U,102012605,2376636,117196,134224,2259440


<div style="border-radius:10px; border:#65647C solid; padding: 15px; background-color: #F8EDE3; font-size:100%; text-align:left">

<h3 align="left"><font color='#7D6E83'><b>🗨️ Comment: </b></font></h3>

When using this method for ranking, it proves inadequate in terms of considering frequency data (total comment count). Let's consider two examples:

1. 6000 likes - 4000 dislikes = 2000
2. 600 likes - 200 dislikes = 400

According to this ranking method, the first example would appear to be a more successful video and expected to make it into the rating list. However, this approach misses the like-dislike ratio, highlighting the need to explore alternative scoring methods.

Another drawback is that this method falls short when calculating up-to-date ratings because older videos often have significantly higher like-dislike counts, further emphasizing its limitations.

In [12]:
def score_average_rating(like, dislike):
    
    if like + dislike == 0:
        
        return 0
    
    return like / (like + dislike)

In [13]:
df["score_average_rating"] = df.apply(lambda x: score_average_rating(x["likes"], x["dislikes"]), axis=1)

In [14]:
df.sort_values("score_average_rating", ascending=False).head(15)

,views,likes,dislikes,comment_count,score_likes_dislikes_diff,score_average_rating
video_id,,,,,,
p7KHGUwqF24,4351,10,0,0,10,1.0000
yirvgC-kMq0,3311,42,0,2,42,1.0000
yq4mgb1PDTI,10345,39,0,0,39,1.0000
Eafi9hqm3OY,5817,22,0,1,22,1.0000
a9i2wTxsxV0,6122,16,0,8,16,1.0000
OmM425PFd3Y,1402,20,0,0,20,1.0000
aHjS9YBXzXU,5454,60,0,4,60,1.0000
aHsfKnrNCG4,3896,9,0,1,9,1.0000
3rhw4KgcvFM,1426,14,0,0,14,1.0000


<div style="border-radius:10px; border:#65647C solid; padding: 15px; background-color: #F8EDE3; font-size:100%; text-align:left">

<h3 align="left"><font color='#7D6E83'><b>🗨️ Comment: </b></font></h3>
    
When this method is used for ranking, it proves insufficient in considering frequency data (total comment count). Let's consider two examples:

1. 6000 likes - 4000 dislikes = 60%
2. 600 likes - 200 dislikes = 75%

In this ranking method as well, when we aim to make a ranking based on the ratio, we expect the second example to be successful and make it into the rating list. However, this time we would be overlooking the frequency data. Total comment count is crucial, and neglecting it may result in a video that garners more community engagement being left behind. Therefore, we should perform a calculation that encompasses both factors - frequency data and ratio.

In [15]:
def wilson_lower_bound(up, down, confidence=0.95): # The confidence level is typically set to 0.95.
    
    """
    Calculate the Wilson Lower Bound Score

    - The lower limit of the confidence interval to be calculated for the Bernoulli parameter p is considered as the WLB score.
    - The calculated score is used for product ranking.
    - Note:
    If the scores are in the range of 1-5, they are marked as 1-3 negative and 4-5 positive, making them suitable for Bernoulli distribution.
    However, this introduces some problems. Therefore, Bayesian average rating should be used.

    Parameters
    ----------
    up: int
        like count
    down: int
        dislike count
    confidence: float
        confidence

    Returns
    -------
    wilson score: float

    """
    n = up + down
    
    if n == 0:
        
        return 0
    
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / n
    
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)

In [16]:
df["wilson_lower_bound"] = df.apply(lambda x: wilson_lower_bound(x["likes"], x["dislikes"]), axis=1)

In [17]:
df.sort_values("wilson_lower_bound", ascending=False).head(15)

,views,likes,dislikes,comment_count,score_likes_dislikes_diff,score_average_rating,wilson_lower_bound
video_id,,,,,,,
p1af9PKM8Eo,43597,5975,4,436,5971,0.9993,0.9983
ONI_06wGbsQ,180546,29537,48,2409,29489,0.9984,0.9978
CFwXUarN-wg,83200,15262,26,881,15236,0.9983,0.9975
6ixU_vdE0Es,280065,20289,38,939,20251,0.9981,0.9974
Q48VduIflPk,2997335,381809,963,23925,380846,0.9975,0.9973
J41qe-TM1DY,7200045,1021328,2716,117897,1018612,0.9973,0.9972
floMqK_yHf8,2412639,382528,1063,27385,381465,0.9972,0.9971
8Jmd7-1quDM,98413,7471,14,313,7457,0.9981,0.9969
X5YJU6_Mfpg,141366,15728,37,1734,15691,0.9977,0.9968


<div style="border-radius:10px; border:#65647C solid; padding: 15px; background-color: #F8EDE3; font-size:100%; text-align:left">

<h3 align="left"><font color='#7D6E83'><b>🗨️ Comment: </b></font></h3>
    
The `wilson_lower_bound` function calculates the Wilson Lower Bound Score, which is used to rank products based on user reviews. This score represents the lower bound of the confidence interval calculated for the Bernoulli parameter "p." "p" represents the probability of success for a binary outcome (for example, positive or negative reviews of a product).

The Wilson Lower Bound Score is a useful metric for ranking products based on user feedback and takes into account the uncertainty associated with a small number of reviews. Products with a higher Wilson Lower Bound Score are ranked higher, indicating a higher level of confidence from users about the product. This method is particularly useful when dealing with binary rating systems (e.g., like or dislike) and can help prevent products with only a few positive ratings from being ranked too high.

In [18]:
us = pd.read_csv("/kaggle/input/youtube-new/USvideos.csv")

In [19]:
us.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


<a id = "4"></a><br>
<p style="font-family: 'Pacifico', cursive; font-weight: bold; letter-spacing: 2px; color: #556B2F; font-size: 160%; text-align: left; padding: 0px; border-bottom: 3px solid">✨Weighted Sorting Score✨</p>

In [20]:
view_score = MinMaxScaler(feature_range=(1, 10)).fit_transform(df[["views"]])
likes_score = MinMaxScaler(feature_range=(1, 10)).fit_transform(df[["likes"]])
dislikes_score = MinMaxScaler(feature_range=(1, 10)).fit_transform(df[["dislikes"]])
comment_count_score = MinMaxScaler(feature_range=(1, 10)).fit_transform(df[["comment_count"]])

df["custom_weight"] = (view_score * 10/100 + \
                       likes_score * 40/100 + \
                       dislikes_score * 30/100 + \
                       comment_count_score * 20/100)

df.sort_values(by="custom_weight", ascending=False).head(10)

,views,likes,dislikes,comment_count,score_likes_dislikes_diff,score_average_rating,wilson_lower_bound,custom_weight
video_id,,,,,,,,
FlsCjmMhFmw,149376127,3093544,1643059,827755,1450485,0.6531,0.6527,7.3245
7C2z4GqqS5E,123010920,5613827,206892,1228655,5406935,0.9645,0.9643,7.0495
QwZT7T-TXT0,37539570,1402578,1674420,1361580,-271842,0.4558,0.4553,6.5495
VYOjWnS4cMY,225211923,5023450,343541,517232,4679909,0.9360,0.9358,6.3591
ffxKSjUwKdU,148689896,3094021,129502,242039,2964519,0.9598,0.9596,4.1071
oWjxSkJpxFU,24286474,1988746,497847,658130,1490899,0.7998,0.7993,4.0452
kTlv5_Bs8aw,36857298,2729292,47896,546100,2681396,0.9828,0.9826,3.6967
6ZfuNTqbHE8,91933007,2625661,53709,350458,2571952,0.9800,0.9798,3.6011
OK3GJ0WIQ8s,23416810,2672431,29088,477233,2643343,0.9892,0.9891,3.4851


<a id = "5"></a><br>
<p style="font-family: 'Pacifico', cursive; font-weight: bold; letter-spacing: 2px; color: #556B2F; font-size: 160%; text-align: left; padding: 0px; border-bottom: 3px solid">✨Weighted Rating✨</p>

In [21]:
def weighted_rating(r, v, M, C):
    return (v / (v + M) * r) + (M / (v + M) * C)

In [22]:
average = (df["likes"] + df["dislikes"]) / df.shape[0]
vote_count = df["likes"] + df["dislikes"]
M = 2500
C = average.mean()
df["degree"] = weighted_rating(average,vote_count,M,C)

df.sort_values(by="degree", ascending=False).head(15)

,views,likes,dislikes,comment_count,score_likes_dislikes_diff,score_average_rating,wilson_lower_bound,custom_weight,degree
video_id,,,,,,,,,
7C2z4GqqS5E,123010920,5613827,206892,1228655,5406935,0.9645,0.9643,7.0495,916.1148
VYOjWnS4cMY,225211923,5023450,343541,517232,4679909,0.9360,0.9358,6.3591,844.6732
FlsCjmMhFmw,149376127,3093544,1643059,827755,1450485,0.6531,0.6527,7.3245,745.4157
ffxKSjUwKdU,148689896,3094021,129502,242039,2964519,0.9598,0.9596,4.1071,507.1753
QwZT7T-TXT0,37539570,1402578,1674420,1361580,-271842,0.4558,0.4553,6.5495,484.1045
kTlv5_Bs8aw,36857298,2729292,47896,546100,2681396,0.9828,0.9826,3.6967,436.8986
p8npDG2ulKQ,29741771,2700800,29341,371864,2671459,0.9893,0.9891,3.3897,429.4909
OK3GJ0WIQ8s,23416810,2672431,29088,477233,2643343,0.9892,0.9891,3.4851,424.9843
6ZfuNTqbHE8,91933007,2625661,53709,350458,2571952,0.9800,0.9798,3.6011,421.4969
